# Preprocesamiento de los datos para poder ser empleados por la _RNA_

Importamos las librerías para el procesamiento de los datos.
Para el proceso de limpieza de los datos inválidos y los outliers emplearemos Pandas.

Por otro lado, en el caso de los datos del pluviómetro para comprobar los valores anómalos emplearemos la librería nativa de peticiones (requests)

In [71]:
import pandas as pd
import requests
import math as m
import os

Creamos las constantes para establecer la dirección en la que se encuentran los ficheros de datos y el directorio en el que se deberá almacenar los resultados

In [72]:
# Rutas de directorios
RAW_FILES_PATH = './data/raw_data'
CLASSIFIED_FILES_PATH = './data/classified_data/'
SENSOR_FILES_DIRECTORY = RAW_FILES_PATH + '/sensores/'
PLUVIOMETER_FILES_DIRECTORY = RAW_FILES_PATH + '/pluviometro/'

# URL de API meteorologica para realizar las comparaciones
WEATHER_API_BASE_URL = 'https://history.openweathermap.org/data/2.5/history/city?lat=41.6966212&lon=-3.9284274&dt={dt}&units=metric&appid={key}'

APIKEY = '9382de5e41593eeb1d830727d6df604d'

Detección los valores atípicos

In [73]:
def outlier_detection(data):
    # Ventana de datos agrupados por dia
    rolling_win = data.groupby(pd.Grouper(freq="24H", key="ts"))

    for _, group in rolling_win:
        # Para cada columna realizamos el analisis por deteccion de valores atipicos
        for col in data.columns:
            if(col != 'ts'):
                # Calculo de la media y la desviacion tipica del grupo
                group_mean = group[col].mean()
                group_std = group[col].std()
                group_median = group[col].median()

                # Definicion del umbral para los valores atipicos (se establece 1 vez la desviacion estandar)
                threshold = 1.4 * group_std

                # Identificacion de los valores atipicos
                outliers = group[(group[col] - group_mean).abs() > threshold]

                data.loc[data['ts'].isin(outliers['ts']), col] = group_median
    

En los ficheros de los datos relacionados con los sensores debemos realizar operaciones diferentes que en el archivo del pluviómetro, por lo que se divide el procesamiento en varias funciones

In [74]:
def main():
    sensor_files = os.listdir(SENSOR_FILES_DIRECTORY)
    sensor_data_processing(sensor_files)
    # pluviometer_data_processing()

def sensor_data_processing(sensor_files):
    for file in sensor_files:

        dataset = pd.read_csv(SENSOR_FILES_DIRECTORY + file)

        # Eliminamos los datos marcados como invalidos
        dataset = dataset[dataset['validez'] != 0]

        # Transformamos los timestamps en datetimes, para posteriormente poder
        # agrupar los datos por dia.
        dataset['ts'] = pd.to_datetime(dataset['ts'], unit="ms")

        # Eliminamos las columnas de fecha, h_C, h_L (por redundancia) y la columna
        # de la bateria (no es esencial en la regresion).
        # Por otro, lado deshechamos la columna de validez (no es necesaria)
        dataset = dataset.drop(['fecha', 'h_C', 'h_L', 'bateria', 'validez'], axis=1)

        # Eliminar nulos (data missing)
        dataset = dataset.dropna()

        # Eliminar outliers
        outlier_detection(dataset)

        dataset['ts'] = pd.to_numeric(dataset['ts'])

        dataset.to_csv(CLASSIFIED_FILES_PATH + file, index = False)

def pluviometer_data_processing():
    pluviometer_files = ["pluviometro.csv"]

    for file in pluviometer_files:
        pluviometer_reader = pd.read_csv(SENSOR_FILES_DIRECTORY + file)

        # Eliminar campos sin valores
        pluviometer_reader = pluviometer_reader.dropna()

        # Eliminar outliers
        

        pluviometer_reader.to_csv(CLASSIFIED_FILES_PATH + file, index = False)

        

In [75]:
if __name__ == "__main__":
    main()